Once I received the k-mers files from Dmitrii, here I want to explore the following points:
- Read those files: are under processed.zip, hence we have to decompressed them first
- Explore the data: how many k-mers I have per each species -> why only those?
- Create a new match function between two scaffolds: how many k-mers are shared between them?

In [6]:
# Import libraries
import pandas as pd
import feather
import zipfile
from sympy import isprime
import os

import k_mers_metric as kmm
import read_k_mers as rkm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read k-mers

In [2]:
'''# Read feather file of each species
# Only the relaxed_locus file is needed
# Only for k prime

# Path to the folder with the processed files
path = '../Data/Intermediate/k_mers/processed/'

df_list = []

for i in range(1, 101):
    # Check if i is prime
    if isprime(i):
        # Read files of all the folders (species)
        for folder in os.listdir(path):
            try:
                # Read feather file
                df = pd.read_feather(path + '/' + folder + '/' + str(i) + '.relaxed_locus.feather')

                # Add species column and k column
                df['species'] = folder
                df['k'] = i

                # Concatenate all the dataframes
                df_list = pd.concat([df_list, df], ignore_index=True)        

                print('File ' + str(i) + ' of ' + folder + ' read')
                    #logging.info('File ' + str(i) + ' of ' + folder + ' read')
            except:
               pass'''



"# Read feather file of each species\n# Only the relaxed_locus file is needed\n# Only for k prime\n\n# Path to the folder with the processed files\npath = '../Data/Intermediate/k_mers/processed/'\n\ndf_list = []\n\nfor i in range(1, 101):\n    # Check if i is prime\n    if isprime(i):\n        # Read files of all the folders (species)\n        for folder in os.listdir(path):\n            try:\n                # Read feather file\n                df = pd.read_feather(path + '/' + folder + '/' + str(i) + '.relaxed_locus.feather')\n\n                # Add species column and k column\n                df['species'] = folder\n                df['k'] = i\n\n                # Concatenate all the dataframes\n                df_list = pd.concat([df_list, df], ignore_index=True)        \n\n                print('File ' + str(i) + ' of ' + folder + ' read')\n                    #logging.info('File ' + str(i) + ' of ' + folder + ' read')\n            except:\n               pass"

In [3]:
'''import glob
import pandas as pd
from sympy import isprime
import multiprocessing

# Path to the folder with the processed files
path = '../Data/Intermediate/k_mers/processed/'

# Define a function to read a single feather file and return a dictionary
def read_file(filepath):
    try:
        df = pd.read_feather(filepath)
        folder, filename = filepath.split('/')[-2:]
        k = int(filename.split('.')[0])
        return {'species': folder, 'k': k, 'data': df}
    except:
        return None

# Use glob to find all the feather files
filepaths = glob.glob(path + '/**/*.relaxed_locus.feather', recursive=True)

# Use multiprocessing to read the files in parallel
with multiprocessing.Pool() as pool:
    data_dicts = pool.map(read_file, filepaths)

# Filter out any failed reads
data_dicts = [d for d in data_dicts if d is not None]

# Concatenate the dictionaries into a single DataFrame
#df = pd.concat([d['data'].assign(species=d['species'], k=d['k']) for d in data_dicts], ignore_index=True)

print('Finished reading', len(data_dicts), 'rows')'''


"import glob\nimport pandas as pd\nfrom sympy import isprime\nimport multiprocessing\n\n# Path to the folder with the processed files\npath = '../Data/Intermediate/k_mers/processed/'\n\n# Define a function to read a single feather file and return a dictionary\ndef read_file(filepath):\n    try:\n        df = pd.read_feather(filepath)\n        folder, filename = filepath.split('/')[-2:]\n        k = int(filename.split('.')[0])\n        return {'species': folder, 'k': k, 'data': df}\n    except:\n        return None\n\n# Use glob to find all the feather files\nfilepaths = glob.glob(path + '/**/*.relaxed_locus.feather', recursive=True)\n\n# Use multiprocessing to read the files in parallel\nwith multiprocessing.Pool() as pool:\n    data_dicts = pool.map(read_file, filepaths)\n\n# Filter out any failed reads\ndata_dicts = [d for d in data_dicts if d is not None]\n\n# Concatenate the dictionaries into a single DataFrame\n#df = pd.concat([d['data'].assign(species=d['species'], k=d['k']) for 

In [8]:
# The good one

import os
import pandas as pd
from sympy import isprime

# Path to the folder with the processed files
path = '../Data/Intermediate/k_mers/processed/'

# Define a function to read a single feather file and return a dictionary
def read_file(filepath, mode = 'strict'):
    try:
        if mode in filepath:
            df = pd.read_feather(filepath)
            folder, filename = os.path.split(filepath)
            k = int(filename.split('.')[0])
            species = folder.split('/')[-1]
            
            print('File ' + str(k) + ' of ' + species + ' read')
            #logging.info('File ' + str(i) + ' of ' + folder + ' read')

            return {'species_gcf_id': species, 'k': k, 'data': df}
    except:
        return None

def process_folder(path):
    # Read all the feather files and store the data in a list of dictionaries
    data_list = []
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        print(folder_path)
        for filename in os.listdir(folder_path):
            filepath = os.path.join(folder_path, filename)
            k = filename.split('.')[0]
            # Check if it is a number and if it is prime
            if k.isdigit():
                if isprime(int(k)):
                    data_dict = read_file(filepath, mode = 'strict')
                    if data_dict is not None:
                        data_list.append(data_dict)
    return data_list

data_list = rkm.process_folder(path, mode='strict')

In [9]:
# Convert the list of dictionaries into a single DataFrame, including the species and k columns
df_list = pd.concat([d['data'].assign(species_gcf_id=d['species_gcf_id'], k=d['k']) for d in data_list], ignore_index=True)
#df_list = pd.concat([d['data'] for d in data_list], ignore_index=True)
df_list = df_list[['species_gcf_id', 'k', 'k_mer', 'replicon_accession']].rename(columns={'replicon_accession': 'scaffold'})

KeyError: 'data'

In [6]:
'''
Sanity check
df_list['count_k_mers'] = df_list['k_mer'].str.count('_') + 1
df_list.loc[df_list['count_k_mers'].astype(int) != df_list['k'].astype(int), 'species'].unique()'''

In [ ]:
'''
test = pd.read_feather('../Data/Intermediate/k_mers/processed/GCF_004010195.1_ASM401019v2_genomic/2.strict_locus.feather')
test['count_k_mers'] = test['k_mer'].str.count('_') + 1
test.loc[test['count_k_mers'] != 2]
df_list.loc[df_list['count_k_mers'].astype(int) != df_list['k'].astype(int), ['count_k_mers', 'species']].value_counts()'''

### Species name
Correct the species name by searching them the readable ones in the config.yaml file

In [8]:
import yaml

def map_species_to_file():
    # Open config.yaml file
    with open('config.yaml', 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

    # Create a dictionary with the filename and the species name from the config file
    # First, take only the organisms with a filename
    organisms = {k: v for k, v in config['organisms'].items() if v['filename'] is not None}

    # Create the dictionary with the filename as key and the species as value
    file_species = {v['filename']: v['name'] for k, v in organisms.items()}

    return file_species

file_species = map_species_to_file()

In [9]:
# Create a column in the dataframe with the species name
df_list['species'] = df_list['species_gcf_id'].map(file_species)

In [15]:
df_list.to_feather('../Data/Intermediate/k_mers/processed/df_k_mers.feather')

In [11]:
# To read them
df_k_mers = pd.read_feather('../Data/Intermediate/k_mers/processed/df_k_mers.feather')

## Algorithm

Aim: given a specific scaffold, know how many k-mers are shared with the rest of scaffolds.

In [10]:
def count_k_mers(df_k_mers, output_folder= '../Data/Intermediate/k_mers/'):
    # From df_k_mers, create a dataframe with the number of k_mers per scaffold / species
    df_k_mers_scaffold = df_k_mers.groupby(['species', 'scaffold', 'k'], as_index=False).nunique()
    df_k_mers_species = df_k_mers_scaffold.groupby(['species', 'k'], as_index=False).sum()

    df_k_mers_scaffold = df_k_mers_scaffold[['species', 'scaffold', 'k', 'k_mer']].rename(columns = {'k_mer': 'total_k_mers'})
    df_k_mers_species = df_k_mers_species[['species', 'k', 'k_mer']].rename(columns = {'k_mer': 'total_k_mers'})

    # Save them into a feather file
    df_k_mers_scaffold.to_feather(output_folder + 'count_k_mers_scaffold.feather')
    df_k_mers_species.to_feather(output_folder + 'count_k_mers_species.feather')

count_k_mers(df_list)

C:\Users\bertr\AppData\Local\Temp\ipykernel_31248\2548606989.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_k_mers_species = df_k_mers_scaffold.groupby(['species', 'k'], as_index=False).sum()


In [11]:
def shared_k_mers(origin, level, df_k_mers):

    # Get the k-mers of the scaffold or species
    df_origin = df_k_mers[df_k_mers[level] == origin]

    # Get the k-mers shared with the rest of scaffolds or species
    df_shared = df_origin.merge(df_k_mers, on = ['k', 'k_mer'])

    df_shared.rename(columns = {
                        'species_y': 'species',
                        'scaffold_y': 'scaffold'}, inplace = True)

    # Remove the k-mers shared within the same species
    df_shared = df_shared.loc[df_shared['species_x'] != df_shared['species'], ['k', level, 'k_mer']]

    # Get the number of k-mers shared with the rest of scaffolds
    df_common_k_mers = df_shared.groupby(['k', level]).count().reset_index()
    df_common_k_mers = df_common_k_mers[['k', level, 'k_mer']].rename(columns = {'k_mer': 'common_k_mers'})

    return df_common_k_mers
    
def generated_k_mers(origin, level, k_mers_counter):
    # Let's remove the rows with NaN values
    # But then, we will remove the information from the scaffold of interest
    # Let's keep it in a separate dataframe
    k_mers_counter_origin= k_mers_counter[k_mers_counter[level] == origin]

    k_mers_counter = k_mers_counter.loc[k_mers_counter['common_k_mers'].notnull()]

    # Merge the two dataframes, creating a new column containing the number of k-mers generated by the original scaffold
    k_mers_counter = k_mers_counter.merge(k_mers_counter_origin[['k', 'total_k_mers', 'length']], on = ['k'], how = 'left', suffixes = ('', '_origin'))

    # Compute the number of k-mers generated by the two scaffolds
    k_mers_counter['generated_k_mers'] = k_mers_counter['total_k_mers'] + k_mers_counter['total_k_mers_origin'] - k_mers_counter['common_k_mers']

    return k_mers_counter

# Metric computation
def metric_computation(k_mers_counter):
    # Let's keep only with the rows with all the information
    k_mers_counter = k_mers_counter.dropna()

    # Compute the probability of finding a common k-mer
    k_mers_counter['probability'] = k_mers_counter['common_k_mers'] / k_mers_counter['generated_k_mers']

    # First, we want to apply the desired formula:
    # (1 - probability) * (1 - (m / (n*k^2)))
    # Where: m, n are the length of the two scaffolds/species with m being the shortest represented in length and length_origin columns
    k_mers_counter['metric'] = (1 - k_mers_counter['probability']) * (1 - (k_mers_counter[['length', 'length_origin']].apply(min, axis=1) / (k_mers_counter[['length', 'length_origin']].apply(max, axis=1) * k_mers_counter['k']**2)))

    # Leave the columnns of interest, so we can aggregate the metric and sort it by species in any cases
    df_metric = k_mers_counter.drop(['total_k_mers', 'total_k_mers_origin', 'probability', 'common_k_mers', 'generated_k_mers', 'length_origin', 'length'], axis = 1)

    # Do the product of the metric for each scaffold/species among all the k values
    if level == 'scaffold':
        return (df_metric
                .groupby(['species', 'scaffold'], as_index=False)
                .prod()
                .sort_values(by = ['species','metric'], ascending = False))
    elif level == 'species':
        return (df_metric
                .groupby('species', as_index=False)
                .prod()
                .sort_values(by = ['species', 'metric'], ascending = False))

def k_mers_to_metric(origin, level, k_mers_counter, df_length, df_k_mers):
    
    # Adapt the information of the scaffold length to the species level
    #if level == 'species':
    df_length = df_length.groupby([level], as_index=False).sum()

    # Add the length of the scaffold or species to the dataframe
    k_mers_counter = k_mers_counter.merge(df_length[[level, 'length']], on = level, how = 'left')

    # Now, we need to compute the number of k-mers shared from the origin with the rest of elements
    print('Computing shared k-mers for: ' + origin + ' '  + level)
    df_common_k_mers = shared_k_mers(origin, level, df_k_mers)

    k_mers_counter = k_mers_counter.join(df_common_k_mers.set_index(['k', level]), on=['k', level], how='left')

    # Now, we have the total k-mers per scaffols and the number of k-mers shared with the rest of scaffolds, i.e., the intersection between the two groups
    # We want to compute the number of k-mers generated by the two scaffolds
    # We can do this by computing the number of k-mers generated by the two scaffolds and subtracting the number of k-mers shared with the rest of scaffolds
    # Done under generated_k_mers function

    k_mers_counter = generated_k_mers(origin, level, k_mers_counter)

    # Now, we can compute the metric
    return metric_computation(k_mers_counter)

In [ ]:
df_list.columns.difference(['species'])

Index(['k', 'k_mer', 'scaffold', 'species_gcf_id'], dtype='object')

In [13]:
import k_mers_metric as kmm

# Given a specific scaffold, know how many k-mers shared with the rest of scaffolds
origin = 'NC_000001.11'
level = 'scaffold'

k_mers_counter = pd.read_feather('../Data/Intermediate/k_mers/count_k_mers_' + level + '.feather')

# For computing the similarity metric between two scaffolds or species, we need:
# - The probability of finding a common k-mer: coomon k-mers / generated k-mers between two scafffolds or species
# - Length of each scaffold
# - k of k-mer

# First, let us add its length information into the k-mers_counter dataframe
# This is already computed under df_scaffold_length file
try:
    df_length = pd.read_csv('../Data/Intermediate/df_' + level + '_length.csv')
    df_length = (df_length[['Specie', 'Replicon Accession', 'Number of genes']]
                        .rename(columns = {'Specie': 'species', 'Replicon Accession': 'scaffold', 'Number of genes': 'length'}))
except:
    print('Please, check the columns of the file df_scaffold_length.csv')

scaffolds_order = kmm.k_mers_to_metric(origin, level, k_mers_counter, df_length, df_list)    #, 'metric']]

c:\Users\bertr\LRZ Sync+Share\Thesis\Code\k_mers_metric.py:92: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_length = df_length.groupby([level], as_index=False).sum()


Computing shared k-mers for: NC_000001.11 scaffold


c:\Users\bertr\LRZ Sync+Share\Thesis\Code\k_mers_metric.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k_mers_counter['probability'] = k_mers_counter['common_k_mers'] / k_mers_counter['generated_k_mers']
c:\Users\bertr\LRZ Sync+Share\Thesis\Code\k_mers_metric.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k_mers_counter['metric'] = (1 - k_mers_counter['probability']) * (1 - (k_mers_counter[['length', 'length_origin']].apply(min, axis=1) / (k_mers_counter[['length', 'length_origin']].apply(max, a

In [ ]:
scaffolds_order

,species,scaffold,k,metric
11,African clawed frog,NC_054382.1,2,0.916613
5,African clawed frog,NC_054376.1,2,0.869437
17,African clawed frog,NC_054388.1,2,0.868704
9,African clawed frog,NC_054380.1,6,0.861646
13,African clawed frog,NC_054384.1,30,0.857382
16,African clawed frog,NC_054387.1,2,0.827235
10,African clawed frog,NC_054381.1,30,0.823000
15,African clawed frog,NC_054386.1,30,0.817991
12,African clawed frog,NC_054383.1,210,0.799004
0,African clawed frog,NC_054371.1,2,0.798263


In [74]:
df_metric

,species_name,replicon_accession,metric
11,African clawed frog,NC_054382.1,0.916613
5,African clawed frog,NC_054376.1,0.869437
17,African clawed frog,NC_054388.1,0.868704
9,African clawed frog,NC_054380.1,0.861646
13,African clawed frog,NC_054384.1,0.857382
16,African clawed frog,NC_054387.1,0.827235
10,African clawed frog,NC_054381.1,0.823000
15,African clawed frog,NC_054386.1,0.817991
12,African clawed frog,NC_054383.1,0.799004
0,African clawed frog,NC_054371.1,0.798263


In [43]:
k_mers_counter[k_mers_counter['length'].notnull()]

,species_name,replicon_accession,k,total_k_mers,length
46,African clawed frog,NC_054371.1,2,1766,1848.0
47,African clawed frog,NC_054371.1,3,1778,1848.0
48,African clawed frog,NC_054371.1,5,1778,1848.0
49,African clawed frog,NC_054371.1,7,1776,1848.0
50,African clawed frog,NC_054371.1,11,1772,1848.0
...,...,...,...,...,...
5386,Human,NT_187693.1,13,17,45.0
5387,Human,NT_187693.1,17,13,45.0
5388,Human,NT_187693.1,19,11,45.0
5389,Human,NT_187693.1,23,7,45.0


In [ ]:
if k_mers_counter

In [40]:
k_mers_counter

,species_name,replicon_accession,k,total_k_mers,length,common_k_mers,total_k_mers_origin,generated_k_mers,probability
0,Aardvark,NW_006921,2,11830,NaN,163.0,496,12163.0,0.013401
1,Aardvark,NW_006921,3,11702,NaN,102.0,508,12108.0,0.008424
2,Aardvark,NW_006921,5,10970,NaN,74.0,516,11412.0,0.006484
3,Aardvark,NW_006921,7,10266,NaN,62.0,520,10724.0,0.005781
4,Aardvark,NW_006921,11,9018,NaN,43.0,520,9495.0,0.004529
...,...,...,...,...,...,...,...,...,...
648,Zebra Finch,NC_044234,2,159,NaN,4.0,496,651.0,0.006144
649,Zebra Finch,NC_044234,3,158,NaN,3.0,508,663.0,0.004525
650,Zebra Finch,NC_044234,5,156,NaN,1.0,516,671.0,0.001490
651,Zebra Finch,NC_044236,2,144,NaN,1.0,496,639.0,0.001565


In [35]:
df_scaffold_length

,Specie,#Replicon Name,Replicon Accession,Number of genes,Start,Stop
0,Aadvark,MT,NC_002078.1,13,2744,15310
1,Aadvark,Un,NW_006921588.1,129,2349339,39525097
2,Aadvark,Un,NW_006921589.1,169,30547,38336954
3,Aadvark,Un,NW_006921590.1,77,719496,36587654
4,Aadvark,Un,NW_006921591.1,33,8315,35191965
...,...,...,...,...,...,...
37053,Xtropicalis,Un,NW_016689751.1,1,557,1575
37054,Xtropicalis,Un,NW_016689781.1,1,1617,2094
37055,Xtropicalis,Un,NW_016689786.1,1,888,1010
37056,Xtropicalis,Un,NW_016689912.1,1,326,410


In [29]:
k_mers_counter

,species_name,replicon_accession,k,total_k_mers,common_k_mers,total_k_mers_origin,generated_k_mers,probability
0,Aardvark,NW_006921,2,11830,163.0,496,12163.0,0.013401
1,Aardvark,NW_006921,3,11702,102.0,508,12108.0,0.008424
2,Aardvark,NW_006921,5,10970,74.0,516,11412.0,0.006484
3,Aardvark,NW_006921,7,10266,62.0,520,10724.0,0.005781
4,Aardvark,NW_006921,11,9018,43.0,520,9495.0,0.004529
...,...,...,...,...,...,...,...,...
648,Zebra Finch,NC_044234,2,159,4.0,496,651.0,0.006144
649,Zebra Finch,NC_044234,3,158,3.0,508,663.0,0.004525
650,Zebra Finch,NC_044234,5,156,1.0,516,671.0,0.001490
651,Zebra Finch,NC_044236,2,144,1.0,496,639.0,0.001565


For computing the metric we need:
- Probability = coomon k-mers / generated k-mers between two scafffolds / species
- Length of each scaffold
- k of the k-mer

In [27]:
df_metric

,species_name,replicon_accession,k,total_k_mers,common_k_mers,total_k_mers_origin,generated_k_mers,probability
0,Aardvark,NW_006921,6469693230,-1943436426934344576,5.234950e+14,-3313660463278981120,3.835701e+39,1.364796e-25
1,Aardvark,NW_006922,210,4903958184840,9.918000e+03,67608053760,1.588917e+13,6.241989e-10
2,African clawed frog,NC_054371.1,2,1766,4.000000e+00,496,2.258000e+03,1.771479e-03
3,African clawed frog,NC_054372.1,2,1342,4.000000e+00,496,1.834000e+03,2.181025e-03
4,African clawed frog,NC_054374.1,2,1254,1.000000e+00,496,1.749000e+03,5.717553e-04
...,...,...,...,...,...,...,...,...
335,Zebra Finch,NC_044232,2310,1732798217700,1.947456e+06,35156187955200,2.570383e+14,7.576522e-09
336,Zebra Finch,NC_044233,2,207,2.000000e+00,496,7.010000e+02,2.853067e-03
337,Zebra Finch,NC_044234,30,3919032,1.200000e+01,130015488,2.896123e+08,4.143470e-08
338,Zebra Finch,NC_044236,2,144,1.000000e+00,496,6.390000e+02,1.564945e-03


In [ ]:
'''# Get the k-mers of the scaffold
df_scaffold = df_list[df_list['replicon_accession'] == scaffold]

# Get the k-mers of the rest of scaffolds
#df_rest = df_list[df_list['replicon_accession'] != scaffold]

# Get the k-mers shared with the rest of scaffolds
df_shared = df_scaffold.merge(df_list, on = ['k', 'k_mer'])

df_shared.rename(columns = {
                    'species_name_y': 'species_name', 
                    'replicon_accession_y': 'replicon_accession'}, inplace = True)

# Remove the k-mers shared within the same species
df_shared = df_shared.loc[df_shared['species_name_x'] != df_shared['species_name'], ['k', 'species_name', 'replicon_accession', 'k_mer']]
                    
# Get the number of k-mers shared with the rest of scaffolds
df_both = df_shared.groupby(['k', 'species_name', 'replicon_accession'], as_index=False).nunique().sort_values('k_mer', ascending = False)

df_both = df_both[['k', 'species_name', 'replicon_accession', 'k_mer']].rename(columns = {'k_mer': 'common_k_mers'})'''

In [ ]:
'''# Compute the number of k_mers per k per scaffold / species, and the number of k_mers shared with the rest of scaffolds
# Then, do # scaffold k_mers + # match_scaffold k_mers - # shared k_mers to get the number of shared k_mers
def generated_k_mers(scaffold, match_scaffold, df_k_mers):
    # Create a list of tuples of k and k_mers per each scaffold
    k_mers_scaffold = df_k_mers[df_k_mers['replicon_accession'] == scaffold][['k', 'k_mer']]
    k_mers_match_scaffold = df_k_mers[df_k_mers['replicon_accession'] == match_scaffold][['k', 'k_mer']]

    # Get the k-mers generated between the two scaffolds
    generated_k_mers = pd.concat([k_mers_scaffold, k_mers_match_scaffold]).drop_duplicates()

    # Get the number of k-mers generated between the two scaffolds per k
    generated_k_mers = generated_k_mers.groupby(['k'], as_index=False).count()

    generated_k_mers['replicon_accession'] = match_scaffold
    generated_k_mers.rename(columns = {'k_mer': 'total_k_mers'}, inplace = True)

    return generated_k_mers'''

In [ ]:
# Now, we want to compute the total number of k-mers that two scaffolds generate
# One scaffold is given; the other one is the rest of scaffolds appearing in df_shared
matching_scaffolds = df_both['replicon_accession'].unique()

df_total_k_mers = pd.DataFrame()

for match_scaffold in matching_scaffolds:
    k_mers = df_list[(df_list['replicon_accession'] == scaffold) | (df_list['replicon_accession'] == match_scaffold)]
    total_k_mers = k_mers.groupby(['k'], as_index=False).nunique().sort_values('k_mer', ascending = False)
    total_k_mers['replicon_accession'] = match_scaffold
    total_k_mers.rename(columns = {'k_mer': 'total_k_mers'}, inplace = True)

    df_total_k_mers = pd.concat([df_total_k_mers, total_k_mers], ignore_index=True)

KeyboardInterrupt: 

In [ ]:
import numpy as np

for match_scaffold in matching_scaffolds:
    df_list['scaffold1'] = np.where(df_list['replicon_accession'] == scaffold, 1, 0)
    df_list['scaffold2'] = np.where(df_list['replicon_accession'] == match_scaffold, 1, 0)

    df_total = df_list.groupby(['k', 'k_mer'], as_index=False).max().sort_values('k_mer', ascending = False)
    df_total['sum'] = df_total['scaffold1'] + df_total['scaffold2']

    # Just take the k-mers that appear in both scaffolds
    df_total = df_total[df_total['sum'] >= 1]

    # unique is not needed, since are already grouped by k and k_mer
    total_k_mers = df_total.groupby(['k'], as_index=False).count().sort_values('k_mer', ascending = False)
    total_k_mers['replicon_accession'] = match_scaffold
    total_k_mers = total_k_mers[['k_mer']].rename(columns = {'k_mer': 'total_k_mers'}, inplace = True)

    df_total_k_mers = pd.concat([df_total_k_mers, total_k_mers], ignore_index=True)

C:\Users\bertr\AppData\Local\Temp\ipykernel_9244\167985797.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_k_mers = total_k_mers[['k_mer']].rename(columns = {'k_mer': 'total_k_mers'}, inplace = True)
C:\Users\bertr\AppData\Local\Temp\ipykernel_9244\167985797.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_k_mers = total_k_mers[['k_mer']].rename(columns = {'k_mer': 'total_k_mers'}, inplace = True)


In [ ]:
import numpy as np

num_k_mers = {}

for match_scaffold in matching_scaffolds:

    # Create a list of tuples of k and k_mers per each scaffold
    k_mers_scaffold = [tuple(row) for row in df_list[df_list['replicon_accession'] == scaffold][['k', 'k_mer']].values.tolist()]
    k_mers_match_scaffold = [tuple(row) for row in df_list[df_list['replicon_accession'] == match_scaffold][['k', 'k_mer']].values.tolist()]

    # Get the k-mers generates between the two scaffolds	
    generated_k_mers = set(k_mers_scaffold).union(k_mers_match_scaffold)

    # Get the number of k-mers generated between the two scaffolds per k
    generated_k_mers = {k: len(v) for k, v in generated_k_mers}

    # Add the scaffold name to the dictionary
    num_k_mers[match_scaffold] = generated_k_mers


In [ ]:
for match_scaffold in matching_scaffolds:

    # Create a list of tuples of k and k_mers per each scaffold
    k_mers_scaffold = df_list[df_list['replicon_accession'] == scaffold][['k', 'k_mer']]
    k_mers_match_scaffold = df_list[df_list['replicon_accession'] == match_scaffold][['k', 'k_mer']]

    # Get the k-mers generated between the two scaffolds
    generated_k_mers = pd.concat([k_mers_scaffold, k_mers_match_scaffold]).drop_duplicates()

    # Get the number of k-mers generated between the two scaffolds per k
    generated_k_mers = generated_k_mers.groupby(['k'], as_index=False).count()

    generated_k_mers['replicon_accession'] = match_scaffold
    generated_k_mers.rename(columns = {'k_mer': 'total_k_mers'}, inplace = True)

In [ ]:
generated_k_mers

,k,k_mer
0,2,831
1,3,845
2,5,851
3,7,853
4,11,849
5,13,847
6,17,843
7,19,841
8,23,833
9,29,821


In [ ]:
for match_scaffold in matching_scaffolds:

    # Create a list of tuples of k and k_mers per each scaffold
    k_mers_scaffold = df_list[df_list['replicon_accession'] == scaffold][['k', 'k_mer']]
    k_mers_match_scaffold = df_list[df_list['replicon_accession'] == match_scaffold][['k', 'k_mer']]

    # Get the k-mers generated between the two scaffolds
    generated_k_mers = k_mers_scaffold.join(k_mers_match_scaffold, lsuffix='_scaffold', rsuffix='_match_scaffold', how='outer')

    # Get the number of k-mers generated between the two scaffolds per k
    generated_k_mers = generated_k_mers.groupby(['k'], as_index=False).count()

In [ ]:
for match_scaffold in matching_scaffolds:
    # Create a list of tuples of k and k_mers per each scaffold
    k_mers_scaffold = df_list[df_list['replicon_accession'] == scaffold][['k', 'k_mer']]
    k_mers_match_scaffold = df_list[df_list['replicon_accession'] == match_scaffold][['k', 'k_mer']]

    # Get the k-mers generated between the two scaffolds
    generated_k_mers = k_mers_scaffold.join(k_mers_match_scaffold, lsuffix='_scaffold', rsuffix='_match_scaffold', how='outer')

    # Get the number of k-mers generated between the two scaffolds per k
    generated_k_mers = generated_k_mers.groupby(['k_scaffold'], as_index=False).count()

In [ ]:
match_scaffold = 'NZ_CP010000.1'

# Create a list of tuples of k and k_mers per each scaffold
k_mers_scaffold = df_list[df_list['replicon_accession'] == scaffold][['k', 'k_mer']]
k_mers_match_scaffold = df_list[df_list['replicon_accession'] == match_scaffold][['k', 'k_mer']]

# Get the k-mers generated between the two scaffolds
generated_k_mers = k_mers_scaffold.join(k_mers_match_scaffold, lsuffix='_scaffold', rsuffix='_match_scaffold', how='outer')

# Get the number of k-mers generated between the two scaffolds per k
generated_k_mers = generated_k_mers.groupby(['k_scaffold'], as_index=False).count()